# Bayes rule for a discrete variable model

This demo is based on the first chapter of "Model-Based Machine Learning" by Winn. The aim is to solve a murder by building a model and collecting evidence.

We propose a model with two suspects described by variable `m`, each with a prior probability defined by the respective entries of `b`. A conditional probability matrix `A` defines the probability for finding a specific weapon given the murderer.

\begin{align*}
    m &\sim \mathcal{C}at(b)\\
    w &\sim \mathcal{C}at(Am)
\end{align*}

Now assume we observe weapon 1 and are interested in inferring a posterior probability for the murderer.

## Specify the model

In [1]:
using ForneyLab

# Build the generative model
g = FactorGraph()

b = [0.7, 0.3] # Prior probability vector
A = [0.2 0.9; 0.8 0.1] # Left-stochastic matrix for conditional probability

@RV m ~ Categorical(b) # Prior
@RV w ~ Transition(m, A) # Observation model

placeholder(w, :w, dims=(2,)); # Placeholder for observation

In [2]:
ForneyLab.draw(g) # Inspect the generated graph

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8487884114425947453 -->
 
 8487884114425947453 
 
 Cat 
 categorical_1 
 
<!-- 8579942011929588004 -->
 
 8579942011929588004 
 
 clamp_1 
 
<!-- 8487884114425947453--8579942011929588004 -->
 
 8487884114425947453--8579942011929588004 
 
 clamp_1 
 1 out 
 2 p 
 
<!-- 10251141649008186238 -->
 
 10251141649008186238 
 
 placeholder_w 
 
<!-- 859988874849132942 -->
 
 859988874849132942 
 
 T 
 transition_1 
 
<!-- 10251141649008186238--859988874849132942 -->
 
 10251141649008186238--859988874849132942 
 
 w 
 1 out 
 1 out 
 
<!-- 2174495356387383610 -->
 
 2174495356387383610 
 
 clamp_2 
 
<!-- 859988874849132942--8487884114425947453 -->
 
 859988874849132942--8487884114425947453 
 
 m 
 1 out 
 2 in1 
 
<!-- 859988874849132942--2174495356387383610 -->
 
 859988874849132942--2174495356387383610 
 
 clamp_2 
 1 out 
 3 a

## Generate the algorithm

In [3]:
algo = sumProductAlgorithm(m) # Build the algorithm code
source_code = algorithmSourceCode(algo)
eval(Meta.parse(source_code)) # Parse and load the algorithm in scope

println(source_code) # Inspect the algorithm code

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleSPCategoricalOutNP(nothing, Message(Multivariate, PointMass, m=[0.7, 0.3]))
messages[2] = ruleSPTransitionIn1PNP(Message(Multivariate, PointMass, m=data[:w]), nothing, Message(MatrixVariate, PointMass, m=[0.2 0.9; 0.8 0.1]))

marginals[:m] = messages[1].dist * messages[2].dist

return marginals

end

end # block


## Execute the algorithm

In [4]:
data = Dict(:w => [1.0, 0.0]) # Weapon 1 is found

marginals = step!(data); # Execute the algorithm

In [5]:
marginals[:m] # Inspect the posterior belief about the murderer

Cat(p=[0.34, 0.66])
